# **getting coding and intergenic seqeunces**

In [32]:
import gzip
from collections import OrderedDict

In [33]:
#hg38 annotation file, i think from http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/genes/
gtf_file = '/scratch/research/references/hsapiens/hg38/hg38.ncbiRefSeq.gtf.gz'

In [47]:
class gtf_line():
    def __init__(self, l, info_delimiter=";", info_field_delimiter = '='):
        self.seqid, self.source, self.type, self.start, self.end,\
        self.score, self.strand, self.phase, self.attribs = l.split('\t')
        self.start = int(self.start)
        self.end = int(self.end)
    def attribute_dict(self, attributes, info_delimiter="; ", info_field_delimiter = ' '): ###this is fragile
        d = OrderedDict()
        attributes = attributes.split(info_delimiter)
        for kv in attributes:
            k,v = kv.split()
            v = v.strip(';')
            v = v.strip('"')
            d[k]=v
        return d

In [64]:
from Bio import SeqIO

#hg38 human genome from elsewhere in our server 
ref_dict = SeqIO.to_dict(SeqIO.parse(gzip.open('/scratch/research/references/hsapiens/hg38/hg38.fa.gz', 'rt'), 'fasta'))

In [ ]:
bc =0
last_transcript = []
current_ch = ''
intergenic_outfile = open("intergenic.fasta", 'w')
for l in gzip.open(gtf_file, "r"):
    l = l.decode('utf8').strip()
    g = gtf_line(l)
    if g.type != 'transcript':continue
    if g.seqid != current_ch:
        print(g.seqid)
        if len(g.seqid) >5:
            continue
        last_transcript = g
        current_ch = g.seqid
        print(current_ch)
        continue
    #if this new transcript doesn't overlap the last output the intergenic region
    if g.start > last_transcript.end:
        # new non overlapping
        # output intergenic
        seq = ref_dict[g.seqid].seq[int(last_transcript.end)-1: int(g.start)]
        intergenic_outfile.write(">{c}:{s}-{e}\n{seq}\n".format(c=g.seqid,s=last_transcript.end,e=g.start,seq=str(seq)))
        # set last
    last_transcript = g
    

In [78]:
intergenic_outfile.close()

In [81]:
!grep ">" intergenic.fasta |wc -l

30452


In [ ]:
bc =0
last_cds = None
current_ch = ''
cds_outfile = open("cds.fasta", 'w')
for l in gzip.open(gtf_file, "r"):
    l = l.decode('utf8').strip()
    g = gtf_line(l)
    if g.type != 'CDS':continue
    if g.seqid != current_ch:
        if len(g.seqid) >5:
            continue
        print(g.seqid)
        last_cds = g
        current_ch = g.seqid
        print(current_ch)
        continue
    #if this new transcript doesn't overlap the last output the intergenic region
    if g.start > last_cds.end:
        # new non overlapping
        # output intergenic
        seq = ref_dict[g.seqid].seq[g.start-1: g.end]
        cds_outfile.write(">{c}:{s}-{e}\n{seq}\n".format(c=g.seqid,s=g.start,e=g.end,seq=str(seq)))
        # set last
    last_cds = g
    